In [59]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
path = "/gdrive/My Drive/ML:March2020/data/"
data = pd.read_csv(path+"paint_mnist.csv",header=None)
data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
data.shape

(3599, 785)

In [62]:
cols=data.shape[1]
print("colums={}".format(cols))

colums=785


In [63]:
y=data.iloc[:,:1]
X=data.iloc[:,1:cols]

In [64]:
X.shape, y.shape

((3599, 784), (3599, 1))

In [65]:
from sklearn.model_selection import train_test_split
X, x_test, y, y_test = train_test_split(X, y, test_size=0.25, random_state=2)
X.shape, x_test.shape, y.shape, y_test.shape

((2699, 784), (900, 784), (2699, 1), (900, 1))

In [66]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [67]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(2699, 4)

In [68]:
y_onehot[0,:]

array([0., 0., 1., 0.])

In [69]:
input_size = 784
hidden_size = 25
classes = 4

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

In [70]:
params = (np.random.random(size=hidden_size * (input_size + 1) + classes * (hidden_size + 1))  - 0.5) * 0.25
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (classes, (hidden_size + 1))))
theta1.shape, theta2.shape

((25, 785), (4, 26))

In [71]:
def forward_propagate(X, theta1, theta2):
    
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    
    z2 = a1 * theta1.T 

    output_layer_1 = sigmoid(z2)

    a2 = np.insert(output_layer_1, 0, values=np.ones(m), axis=1)
       
    z3 = a2 * theta2.T 

    y_pred= sigmoid(z3)
    
    return a1, z2, a2, z3, y_pred

In [72]:
a1, z2, a2, z3, y_pred = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, y_pred.shape

((2699, 785), (2699, 25), (2699, 26), (2699, 4), (2699, 4))

In [73]:
def cost(params, input_size, hidden_size, classes, X, y, reg_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (classes, (hidden_size + 1))))
    
    a1, z2, a2, z3, y_pred = forward_propagate(X, theta1, theta2)
    
    cost = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(y_pred[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - y_pred[i,:]))
        cost += np.sum(first_term - second_term)
    
    cost = cost / m
    
    cost += (float(reg_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return cost

In [74]:
reg_rate = 1
cost(params, input_size, hidden_size, classes, X, y_onehot, reg_rate)

2.9975488784630655

In [75]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [76]:
def backprop(params, input_size, hidden_size, classes, X, y, reg_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (classes, (hidden_size + 1))))
    
    a1, z2, a2, z3, y_pred = forward_propagate(X, theta1, theta2)
    
    cost = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(y_pred[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - y_pred[i,:]))
        cost += np.sum(first_term - second_term)
    
    cost = cost / m
    
    cost += (float(reg_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        hypred_t = y_pred[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = hypred_t - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * reg_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * reg_rate) / m
    
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return cost, grad

In [77]:
cost, grad = backprop(params, input_size, hidden_size, classes, X, y_onehot, reg_rate)
cost

2.9975488784630655

In [78]:
from scipy.optimize import minimize
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, classes, X, y_onehot, reg_rate), 
                method='TNC', jac=True, options={'maxiter': 750})
fmin

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


     fun: 0.5705249893190473
     jac: array([ 1.97225582e-04,  1.38191004e-05,  9.38152652e-06, ...,
       -1.22629895e-03, -9.81198065e-04, -3.55640522e-04])
 message: 'Max. number of function evaluations reached'
    nfev: 750
     nit: 153
  status: 3
 success: False
       x: array([-0.19089031,  0.03729775,  0.02532074, ...,  0.28814968,
        0.48249599,  1.15151013])

In [79]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (classes, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

y_pred = np.array(np.argmax(h, axis=1))
y_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


array([[2],
       [3],
       [1],
       ...,
       [0],
       [1],
       [1]])

In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y)

0.9277510188958874

In [81]:
x_test.shape, X.shape

((900, 784), (2699, 784))

In [82]:
x_test = np.matrix(x_test)

In [83]:
t_a1, t_z2, t_a2, t_z3, t_h = forward_propagate(x_test, theta1, theta2)
t_y_pred = np.array(np.argmax(t_h, axis=1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, t_y_pred)

0.8622222222222222

In [85]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2', 'class 3']
print(classification_report(y_test, t_y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.91      0.93      0.92       218
     class 1       0.88      0.85      0.87       238
     class 2       0.84      0.85      0.85       219
     class 3       0.81      0.82      0.81       225

    accuracy                           0.86       900
   macro avg       0.86      0.86      0.86       900
weighted avg       0.86      0.86      0.86       900

